In [ ]:
import os
import torch
import numpy as np

# CODE FILES HERE
from model_params import get_model_data_dcgan
from models.dcgan import Dcgan, Generator, Discriminator
from solver import Solver
from directories import Directories
from dataloader import DataLoader
from plots import plot_losses, plot_z_samples, plot_celeba, plot_animation_celeba
from sampling import dcgan_sampling

# SETTINGS HERE
os.environ['CUDA_LAUNCH_BLOCKING'] = "1" # to see the CUDA stack
%matplotlib inline
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2
# supress cluttering warnings in solutions
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# setting device on GPU if available, else CPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

# Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')

In [ ]:
dataset_arg = "celeba"
data = get_model_data_dcgan(dataset_arg)

In [ ]:
directories = Directories("dcgan", dataset_arg.lower(), data["z_dim"], make_dirs=True)
data_loader = DataLoader(directories, data["batch_size"], dataset_arg.lower())

In [ ]:
x = next(iter(data_loader.train_loader))
plot_celeba(x[0])

In [ ]:
model = Dcgan(data_loader.input_dim, data["z_dim"])
generator = Generator(data["z_dim"], data_loader.img_dims[0])
discriminator = Discriminator(data_loader.img_dims[0])
solver = Solver(model, generator, discriminator, data["epochs"], data_loader, data["optimizer_G"],
                data["optimizer_D"], data["optim_config_G"], data["optim_config_D"], save_model_state=False)
solver.main()

In [ ]:
# Plotting g and d losses for all epochs
plot_losses(solver, solver.train_loss_history["g_loss"], solver.train_loss_history["d_loss"])

In [ ]:
# Plot samples from the latent space.
samples = dcgan_sampling(generator, solver.model.z_dim, 100)
plot_z_samples(samples, solver.data_loader.directories.make_dirs,
               solver.data_loader.directories.result_dir, solver.data_loader.dataset)

In [ ]:
plot_animation_celeba(solver.data_loader.directories.result_dir, solver.train_gen_imgs,\
                      solver.data_loader.directories.make_dirs)